In [1]:
#pip install langchain-pinecone pinecone-client

 - 임베딩 : 텍스트를 숫자 배열로 변환. 임베딩 모델을 통해서 실행.

 - 벡터 : 임베딩 결과로 만들어진 숫자 배열

 - 벡터 DB : 벡터 값들을 저장하고 벡터로 조회할 수 있습니다.

In [3]:
import os
pinecone_key = os.getenv('PINECONE_API_KEY')
pinecone_key

'pcsk_6bYEch_C6MyWm9ftZgVbruvc9Tb4j8Ssd8SWZLhB5DYBTEG5orqgMPxZkh5SQ71Uew6fNe'

In [6]:
# 벡터DB 파인콘 객체 초기화
from pinecone import Pinecone
pine = Pinecone(api_key=pinecone_key)
# 인덱스 : 관계형 db의 '테이블' 또는 스키마 개념
# 인덱스를 웹사이트에서 생성 : 이름은 test-embedding-3d

In [8]:
#test-embedding-3d 인덱스를 사용하기 위해 객체를 생성
index = pine.Index("test-embedding-3d")
#인덱스 기본 정보 확인
index.describe_index_stats()

{'dimension': 3,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {},
 'total_vector_count': 0,
 'vector_type': 'dense'}

In [ ]:
# 데이터(벡터) 업서트
index.upsert(
    vectors=[
        {
            "id": "vec1", #id : 식별값 ( 필수)
            "values": [1.0, 1.5, 2.0], # 벡터(임의로 부여함.)
            "metadata": {"genre": "drama"} # 벡터와 관련된 텍스트 값
        }, {
            "id": "vec2",
            "values": [2.0, 1.0, 0.5],
            "metadata": {"genre": "action"}
        }, {
            "id": "vec3",
            "values": [0.1, 0.3, 0.5],
            "metadata": {"genre": "drama"}
        }, {
            "id": "vec4",
            "values": [1.0, 2.5, 3.5],
            "metadata": {"genre": "action"}
        }, {
            "id": "vec5",
            "values": [3.0, 1.2, 1.3],
            "metadata": {"genre": "action"}
        }, {
            "id": "vec6",
            "values": [0.3, 1.1, 2.5],
            "metadata": {"genre": "drama"}
        }
    ],
    namespace="embedding-3d-ns1"
)

{'upserted_count': 6}

In [15]:
#4. 특정 벡터값으로 질의(쿼리,조회)하기 : 텍스트로 조회하지 않습니다.
response = index.query(
  namespace = 'embedding-3d-ns1',
  vector=[0.5,0.3,0.1], #조회할 값
  top_k=3,
  include_values = True,
  include_metadata = True
)
print(response)
#↪결과 값 중에 score는 유사도 : 대체로 0~1

{'matches': [{'id': 'vec2',
              'metadata': {'genre': 'action'},
              'score': 0.996488571,
              'values': [2.0, 1.0, 0.5]},
             {'id': 'vec5',
              'metadata': {'genre': 'action'},
              'score': 0.966202915,
              'values': [3.0, 1.2, 1.3]},
             {'id': 'vec1',
              'metadata': {'genre': 'drama'},
              'score': 0.723038375,
              'values': [1.0, 1.5, 2.0]}],
 'namespace': 'embedding-3d-ns1',
 'usage': {'read_units': 1}}


In [33]:
#텍스트 문장으로 벡터 만들기
from pinecone import ServerlessSpec
index_name = "quickstart"

for idx in pine.list_indexes():
  if idx.name == index_name:
    pine.delete_index(index_name)
#함수로 인덱스 만들기
pine.create_index(
  name=index_name,
  dimension=1024, #임베딩 값의 차원
  spec=ServerlessSpec(
    cloud="aws",
    region="us-east-1"

  )
)



{
    "name": "quickstart",
    "metric": "cosine",
    "host": "quickstart-87mf77p.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 1024,
    "deletion_protection": "disabled",
    "tags": null
}

In [18]:
pine.list_indexes()

[
    {
        "name": "quickstart",
        "metric": "cosine",
        "host": "quickstart-87mf77p.svc.aped-4627-b74a.pinecone.io",
        "spec": {
            "serverless": {
                "cloud": "aws",
                "region": "us-east-1"
            }
        },
        "status": {
            "ready": true,
            "state": "Ready"
        },
        "vector_type": "dense",
        "dimension": 1024,
        "deletion_protection": "disabled",
        "tags": null
    },
    {
        "name": "test-embedding-3d",
        "metric": "cosine",
        "host": "test-embedding-3d-87mf77p.svc.aped-4627-b74a.pinecone.io",
        "spec": {
            "serverless": {
                "cloud": "aws",
                "region": "us-east-1"
            }
        },
        "status": {
            "ready": true,
            "state": "Ready"
        },
        "vector_type": "dense",
        "dimension": 3,
        "deletion_protection": "disabled",
        "tags": null
    }
]

In [30]:
data = [
    {"id": "vec1", "text": "사과는 달콤하고 아삭한 식감으로 유명한 인기 있는 과일입니다."},
    {"id": "vec2", "text": "애플이라는 기술 회사는 아이폰과 같은 혁신적인 제품으로 유명합니다."},
    {"id": "vec3", "text": "많은 사람들이 건강한 간식으로 사과를 즐겨 먹습니다."},
    {"id": "vec4", "text": "애플 주식회사는 세련된 디자인과 사용자 친화적인 인터페이스로 기술 산업을 혁신했습니다."},
    {"id": "vec5", "text": "하루에 사과 하나면 의사를 멀리할 수 있다는 속담이 있습니다."},
    {"id": "vec6", "text": "애플 컴퓨터 회사는 1976년 4월 1일 스티브 잡스, 스티브 워즈니악, 로널드 웨인에 의해 파트너십으로 설립되었습니다."}
]

In [31]:
# 임베딩 -> upsert
embeddings = pine.inference.embed(
  model='multilingual-e5-large',
  inputs=[d['text'] for d in data],
  parameters={"input_type":"passage","truncate":"END"}
  
)

In [32]:
print(embeddings[0])
print(len(embeddings[0].values))

{'vector_type': dense, 'values': [0.0302886962890625, -0.0204620361328125, ..., -0.0301666259765625, 0.024078369140625]}
1024


In [34]:
# 인덱스를 초기화합니다.
index = pine.Index(index_name)

# 벡터 리스트를 초기화합니다.
vectors = []
for d, e in zip(data, embeddings):
    # 각 데이터와 임베딩을 결합하여 벡터를 생성합니다.
    vectors.append({
        "id": d['id'],
        "values": e['values'],
        "metadata": {'text': d['text']}
    })

# 벡터를 인덱스에 업서트(upsert)합니다. quickstart_ns1 네임스페이스에 벡터를 추가합니다.
index.upsert(
    vectors=vectors,
    namespace="quickstart_ns1"
)

{'upserted_count': 6}

In [38]:
#query = "애플이라는 기술 회사에 대해 알려주세요."
query = "사과는 맛이 어떤지 알려주세요."
# 쿼리 텍스트를 임베딩 벡터로 변환합니다.
embedding = pine.inference.embed(
  model="multilingual-e5-large",
  inputs=[query],
  parameters={
    "input_type":"query" #입력 타입을 쿼리로 설정합니다.
  }
)

#ㅊ첫 번째 임베딩 벡터를 출력합니다.
print(embedding[0])

{'vector_type': dense, 'values': [0.004001617431640625, -0.01073455810546875, ..., -0.0262451171875, 0.0196990966796875]}


In [39]:
response=index.query(
  namespace='quickstart_ns1',
  vector=embedding[0].values,
  top_k=3,
  include_values=False,
  include_metadata=True
)
print(response)


{'matches': [{'id': 'vec1',
              'metadata': {'text': '사과는 달콤하고 아삭한 식감으로 유명한 인기 있는 과일입니다.'},
              'score': 0.89238739,
              'values': []},
             {'id': 'vec3',
              'metadata': {'text': '많은 사람들이 건강한 간식으로 사과를 즐겨 먹습니다.'},
              'score': 0.848899782,
              'values': []},
             {'id': 'vec5',
              'metadata': {'text': '하루에 사과 하나면 의사를 멀리할 수 있다는 속담이 있습니다.'},
              'score': 0.808141351,
              'values': []}],
 'namespace': 'quickstart_ns1',
 'usage': {'read_units': 1}}
